# Speaking about family: differences in speech patterns across demographics 

This notebook contains all of the analyses for Group 13's final project in Linguistics 1. 

## Project Proposal

1. **Issue/Behavior of interest** We are interested in exploring how conversations about family differ across gender/age/other demographics. Perhaps there is a gender difference in the types of words surrounding family conversations, or perhaps the gender makeup of the conversations (man-to-man vs. man-to-woman vs. woman-to-woman) result in significant differences in the words. These are the general questions we are interested in pursuing.

2. **Research Question** Are there any notable differences in the keywords or the sentiment used by male speakers and female speakers in conversations about family?

3. **Proposed Methods** Since we are interested specifically about conversations about family, we plan to preprocess the data to only specifically select conversations with a listed “Theme/Topic” of Family, or conversations where family related keywords could appear. Within this subset of the data, we plan to compare 1) the frequency of particular content words and 2) the sentiment of content words across different speaker populations (male, female, young, old). Comparing the frequency of content words will be straightforward; doing sentiment analysis will be slightly more complicated, but open source sentiment analysis software (NLTK) will simplify the task immensely. As a first past, we’ll look at whether females/males speak more/less positively while speaking about family.

4. **Further Questions** Is there a dependence on the speaker’s content words (either frequency or sentiment) on the listener? Are there interaction effects between the age and sex of a speaker on the sentiment of their speech? 


## Data Preprocessing

The raw data contains 17683 entries. Each entry corresponds to a single word, and a number of features related to the setting in which the word was spoken:
* Cafe Name
* Time of Day
* Speaker Age and Gender
* Listener 1 and 2 Age and Gender
* Theme/Topic
* Formality (Informal, Neutral, Formal)

### Filtering Theme/Topic
For our study, we are interested solely in entries whose Theme/Topic contains the word family. Thus, we filtered the entries to include only those who's 'Theme/Topic' column contained the word 'Family'. 

Of these remaining entries, one conversation was labeled as 'General discussion among family'; we discarded all such entries because we are interested in conversations *about* family, not just *among* family.

### Fixing Ages
Many of the recorded ages included ranges rather than single numbers (e.g. '18-25', '30s'). In these cases, we approximate these ages with the floored mean of the range (e.g. 21, 34).

### Phrases to words
Despite clear instruction, some students failed to understand that the 'Content Words (one word, not phrases)' column was to contain one word, not phrases. In cases where phrases were recorded, we either 1) reduced the phrase to a content word 2) split the phrase into multiple content words or 3) deleted the entry if the phrase contained no content words.

The CSV with the raw data can be found [here](family_raw_data.csv), and the cleaned data is [here](family_clean_data.csv)